<a href="https://colab.research.google.com/github/Bray-Nyagwoka/ISTG6010-2025/blob/main/Week5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import random

class Result:
    HIT = "Hit"
    MISS = "Miss"
    SUNK = "Sunk"
    INVALID = "Invalid"
    ALREADY_TAKEN = "Already Taken"

class Ship:
    """Represents a ship on the grid."""
    def __init__(self, name, size):
        self.name = name
        self.size = size
        self.positions = []  # List of (row, col) tuples
        self.hits = set()

    def place(self, start_row, start_col, horizontal):
        self.positions.clear()
        for i in range(self.size):
            row = start_row
            col = start_col
            if horizontal:
                col += i
            else:
                row += i
            self.positions.append((row, col))

    def register_hit(self, position):
        if position in self.positions:
            self.hits.add(position)
            return True
        return False

    def is_sunk(self):
        return set(self.positions) == self.hits

class Grid:
    """Represents a player's grid, with ships and shots."""
    def __init__(self, size=10):
        self.size = size
        self.ships = []
        self.shots = set()

    def is_valid_position(self, ship):
        """Ensure the ship's positions are within bounds and not overlapping."""
        for pos in ship.positions:
            r, c = pos
            if not (0 <= r < self.size and 0 <= c < self.size):
                return False
            for existing in self.ships:
                if pos in existing.positions:
                    return False
        return True

    def add_ship(self, ship):
        if self.is_valid_position(ship):
            self.ships.append(ship)
            return True
        return False

    def receive_shot(self, position):
        if position in self.shots:
            return ShotResult.ALREADY_TAKEN
        self.shots.add(position)
        for ship in self.ships:
            if ship.register_hit(position):
                if ship.is_sunk():
                    return ShotResult.SUNK
                return ShotResult.HIT
        return ShotResult.MISS

    def all_ships_sunk(self):
        return all(ship.is_sunk() for ship in self.ships)

class Player:
    """Represents a player, human or AI."""
    def __init__(self, name):
        self.name = name
        self.grid = Grid()

    def place_ships_randomly(self, ship_definitions):
        for name, size in ship_definitions:
            placed = False
            while not placed:
                horizontal = random.choice([True, False])
                row = random.randint(0, self.grid.size - 1)
                col = random.randint(0, self.grid.size - 1)
                ship = Ship(name, size)
                ship.place(row, col, horizontal)
                placed = self.grid.add_ship(ship)

    def take_turn(self, opponent_grid):
        while True:
            try:
                user_input = input(f"{self.name}, enter shot (row,col): ")
                row, col = map(int, user_input.strip().split(','))
                if not (0 <= row < opponent_grid.size and 0 <= col < opponent_grid.size):
                    raise ValueError
                result = opponent_grid.receive_shot((row, col))
                print(f"{self.name}'s shot at ({row},{col}): {result}")
                return result
            except ValueError:
                print("Invalid input. Enter coordinates like 2,3.")

class Game:
    """Manages game state and flow."""
    def __init__(self):
        self.ship_definitions = [("Destroyer", 2), ("Submarine", 3), ("Battleship", 4)]
        self.player = Player("Player")
        self.computer = Player("Computer")

    def setup(self):
        print("Placing player ships randomly...")
        self.player.place_ships_randomly(self.ship_definitions)
        print("Placing computer ships randomly...")
        self.computer.place_ships_randomly(self.ship_definitions)

    def play(self):
        print("Game start!")
        turn = 0
        while True:
            if turn % 2 == 0:
                result = self.player.take_turn(self.computer.grid)
            else:
                while True:
                    row = random.randint(0, self.computer.grid.size - 1)
                    col = random.randint(0, self.computer.grid.size - 1)
                    if (row, col) not in self.player.grid.shots:
                        break
                result = self.player.grid.receive_shot((row, col))
                print(f"Computer's shot at ({row},{col}): {result}")

            if self.computer.grid.all_ships_sunk():
                print("Player wins!")
                break
            elif self.player.grid.all_ships_sunk():
                print("Computer wins!")
                break
            turn += 1


In [4]:
# Start the Battleship game
game = Game()
game.setup()
game.play()


Placing player ships randomly...
Placing computer ships randomly...
Game start!
Player, enter shot (row,col): 2
Invalid input. Enter coordinates like 2,3.
Player, enter shot (row,col): 2,3
Player's shot at (2,3): Miss
Computer's shot at (7,9): Miss


KeyboardInterrupt: Interrupted by user